<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/013_AgentZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 🧠 1. **Understand What an AI Agent Is**

An **AI agent** is a system that perceives its environment, makes decisions, and acts to achieve a goal. This often includes:

* **Perception** (input): Taking in data (text, images, etc.).
* **Reasoning**: Deciding what to do.
* **Action**: Executing tasks (e.g., sending messages, querying APIs, moving in a game).

Examples: ChatGPT agents, game bots, customer support agents, trading bots, etc.

---

### 🛠 2. **Core Components of an AI Agent**

* **Environment**: The world or interface the agent operates in (e.g., web, terminal, game).
* **State**: What the agent knows about the environment.
* **Policy/Logic**: How it decides what to do (often powered by LLMs or ML models).
* **Actuator**: How it executes its actions.
* **Memory**: How it remembers previous states/interactions (optional, but powerful).

---

### 📚 3. **Technologies & Tools**

Depending on your goals, here are the main tools you might use:

| Purpose          | Tools                                                                |
| ---------------- | -------------------------------------------------------------------- |
| LLM APIs         | OpenAI, Anthropic, Cohere                                            |
| Memory           | LangChain, LlamaIndex, custom vector DB                              |
| Execution        | Python, Node.js                                                      |
| Planning         | LangGraph, ReAct, AutoGPT-style architectures                        |
| UI               | Streamlit, Gradio, custom frontend                                   |
| Agents Framework | LangChain Agents, OpenAI Functions, AutoGen, CrewAI, Semantic Kernel |

---

### 🧪 4. **Build a Simple AI Agent**

We can start by building a **basic AI assistant** that can:

* Take a user request.
* Decide what tool/API to use (e.g., calculator, search).
* Return an answer.

Would you like:

* A **code example** to build a basic agent?
* A walkthrough of **LangChain/OpenAI function calling**?
* A **project idea** you can build step by step?




##Pip Installs

In [ ]:
!pip install -q transformers accelerate huggingface_hub litellm python-dotenv

Let’s build a **basic AI agent from scratch** using OpenAI’s API. This will teach you:

1. **What makes an agent an agent**
2. **How it processes information and makes decisions**
3. **The core loop: perceive → reason → act**

---

## 🧠 Part 1: What Is an AI Agent?

An **AI agent** has:

| Component             | Role                                                        |
| --------------------- | ----------------------------------------------------------- |
| **Perception**        | Understands its inputs (e.g., user text)                    |
| **Reasoning**         | Chooses what to do next (often using LLM)                   |
| **Action**            | Performs an action (e.g., call a function, return a result) |
| **Memory** (optional) | Remembers past events for context                           |

---

## 🛠️ Part 2: Minimal AI Agent (Python + OpenAI)

Let’s build a simple Python agent that:

* Takes a user request
* Decides what kind of tool to use (calculator or info lookup)
* Performs the action
* Returns the result

We’ll define two tools:

1. **Calculator**
2. **Dummy search** (you can replace it with real web/API later)

## 🧠 What This Teaches

* The **agent’s brain** is the LLM (GPT-4), deciding what to do.
* The agent doesn't directly do the math or search — it delegates to **tools**.
* This is an example of a **Tool-using Agent** pattern — a key building block in AI agents.




## Agent Zero


### ✅ OpenAI API Basics (How It Works)

When you use a model like `gpt-4o-mini` via the **OpenAI Python SDK**:

* You **don’t download** the model.
* Your code sends a **request** (via API) to OpenAI's servers.
* OpenAI runs the model on their **GPU infrastructure**.
* You get a **response** (text, functions, etc.) back via the API.

Think of it like this:

```plaintext
Your Code  →  API Request →  OpenAI GPUs →  Model Output →  Your Code
```

So your machine does none of the heavy lifting — which is why it works from a laptop, Raspberry Pi, or serverless function the same way.

---

### 💰 Why Use `gpt-4o-mini`?

* **Much cheaper** (ideal for prototyping agents).
* **Still quite capable** for reasoning, parsing, tool calling.
* **Lower latency** (often faster responses).

It's a perfect fit for early agent experimentation — then you can scale up to a bigger model once your system works well or needs more capability.



In [ ]:
from openai import OpenAI
import math
import os
import json
import logging
from dotenv import load_dotenv
import litellm
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load token from .env.
load_dotenv("/content/API_KEYS.env", override=True)

# 1️⃣  Set your key (or rely on the env var).
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ----- tools ----------------------------------------------------
def calculator_tool(expr: str) -> str:
    try:
        # DON'T use eval with un-trusted input in production!
        return f"The result is {eval(expr, {'__builtins__': None}, math.__dict__)}"
    except Exception as e:
        return f"Calculator error: {e}"

def search_tool(query: str) -> str:
    # Placeholder for a real search call
    return f"Search result for '{query}': [dummy data]"

tools = {"calculator": calculator_tool, "search": search_tool}

# ----- agent core ----------------------------------------------
SYSTEM_PROMPT = """
You are an intelligent agent with two tools:
• calculator — for arithmetic/expression evaluation
• search     — for factual look-ups

When you decide on an action, reply **only** in this format:
Tool: <tool_name>
Input: <input_for_tool>
"""

def agent_core(user_input: str) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_input},
    ]

    llm_reply = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.0,   # deterministic planning
    ).choices[0].message.content.strip()

    print("\nAgent reasoning:\n", llm_reply)

    # very simple parser for the two-line format
    try:
        tool_name = llm_reply.splitlines()[0].split(":",1)[1].strip()
        tool_input = llm_reply.splitlines()[1].split(":",1)[1].strip()
        if tool_name not in tools:
            return f"Unknown tool '{tool_name}'."
        return tools[tool_name](tool_input)
    except Exception as e:
        return f"Failed to parse tool call: {e}"

# ----- CLI loop -------------------------------------------------
if __name__ == "__main__":
    print("Type 'exit' or press Ctrl+C to quit.")
    try:
        while True:
            user_in = input("\nYou: ")
            if user_in.lower() in {"exit", "quit"}:
                print("Goodbye!")
                break
            print("Agent:", agent_core(user_in))
    except KeyboardInterrupt:
        print("\n\nInterrupted by user. Goodbye!")

Type 'exit' or press Ctrl+C to quit.

You: What is the square root of 34876?

Agent reasoning:
 Tool: calculator
Input: sqrt(34876)
Agent: The result is 186.75117134840144

You: What is 2 +7?

Agent reasoning:
 Tool: calculator
Input: 2 + 7
Agent: The result is 9

You: exit
Goodbye!


## Common Usage of Agents

Here’s a well-researched look at where AI agents are **actually being used today**, what problems they solve, and what seasoned developers find **most practical**:

---

## 🔍 1. Customer Service & Support

* Agents automate common inquiries — like shipping status, password resets, returns — freeing humans to handle complex issues ([botpress.com][1]).
* Enterprises like Salesforce and ServiceNow report major efficiency gains: ServiceNow reduced case-handling time by \~52% ([businessinsider.com][2]).

---

## 📈 2. Sales & Marketing Automation

* From lead generation and email follow‑ups to personalized campaigns — agents can segment audiences, outreach, and track responses ([botpress.com][1]).
* Dun & Bradstreet uses Gemini-based agents to craft emails; Cognizant employs them to assist legal teams in drafting contracts ([cloud.google.com][3]).

---

## 🔧 3. IT & DevOps

* Automating ticket triage, troubleshooting (e.g., network issues), generating code or tests ([convergetp.com][4]).
* Developers use agents to refactor code, generate tests, and streamline workflows—though there’s pushback about quality/safety ([ibm.com][5]).

---

## 🚚 4. Logistics & Transportation

* Agents coordinate dispatching fleets, reroute deliveries in real-time based on delays or traffic, and handle predictive maintenance ([ibm.com][6]).

---

## 🏥 5. Healthcare & Finance

* In finance, agents execute algorithmic trades (like JPMorgan’s LOXM), handle underwriting, detect fraud ([en.wikipedia.org][7]).
* In healthcare, agents support diagnostics (e.g. eye disease detection from scans), manage workflow like appointment scheduling ([en.wikipedia.org][7]).

---

## 🏭 6. Manufacturing & Maintenance

* Siemens uses agents for real-time sensor analysis and predictive maintenance, cutting downtime by \~25% ([en.wikipedia.org][7]).

---

## 🔐 7. Cybersecurity

* Agents monitor logs and network traffic, automatically flagging and responding to threats or vulnerabilities .

---

### 🧠 Expert Developer Perspective

* Many say agents are most **useful for narrow, repetitive tasks with clear goals** — not flashy multi-step autonomy ([en.wikipedia.org][8]).
* Apple, Amazon have pulled back full-agent ambitions, opting for human-in-the-loop and tightly controlled workflows .

---

## 🎯 TL;DR: Practical Use-Cases

1. **Customer service bots** that do work (not just chat).
2. **Sales/marketing automation** — outreach, lead scoring.
3. **DevOps/code agents** — tests, ticketing, refactoring.
4. **Logistics dispatchers** — fleets, routing, alerts.
5. **Predictive maintenance** & manufacturing monitoring.
6. **Finance operations** — trading bots, underwriting.
7. **Security defense** — threat detection & response.

---

### ✅ Why This Matters to You

* **Start narrow & specific** — pick a real problem where time is wasted or manual work is consistent.
* **Skip hype** — avoid multi-agent dream systems at first.
* **Human-in-the-loop is key** — oversight, validation, guardrails.
* **Measure ROI** — time saved, customer satisfaction, cost reductions.

